<a href="https://colab.research.google.com/github/akshayakannan05/akshayakannan05/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk flask ngrok

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

import json
import random
import re
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
            "responses": ["Hello! How can I help you today?", "Hi there! How may I assist you?"]
        },
        {
            "tag": "order_status",
            "patterns": ["Where is my order?", "Track my order", "Order status"],
            "responses": ["Can you please provide your order ID?", "Sure, please share your order ID to track."]
        },
        {
            "tag": "return_policy",
            "patterns": ["Return policy", "How to return an item?", "I want to return my product"],
            "responses": ["Our return policy allows returns within 30 days. Would you like to initiate a return?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later"],
            "responses": ["Goodbye! Have a great day!", "See you again soon!"]
        }
    ]
}

def clean_text(text):
    words = nltk.word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join(words)

def chatbot_response(user_input):
    user_input = clean_text(user_input)

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # Clean pattern for better matching
            cleaned_pattern = clean_text(pattern)
            if re.search(r'\b' + re.escape(cleaned_pattern) + r'\b', user_input):
                return random.choice(intent['responses'])

    return "I'm sorry, I didn't understand that. Could you please rephrase?"

print("Chatbot: Hello! I am here to assist you. Type 'quit' to exit.\n")

while True:
    user_message = input("You: ")
    if user_message.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_message)
    print("Chatbot:", response)

from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app runs

# HTML Template
html_template = """
<!DOCTYPE html>
<html>
<head>
  <title>Customer Service Chatbot</title>
  <style>
    body { font-family: Arial, sans-serif; padding: 20px; }
    #chatbox { border: 1px solid #ccc; padding: 10px; width: 400px; height: 300px; overflow-y: scroll; }
    .user { color: blue; }
    .bot { color: green; }
  </style>
</head>
<body>
  <h2>Customer Service Chatbot</h2>
  <div id="chatbox"></div>
  <input type="text" id="userInput" placeholder="Type your message here..." />
  <button onclick="sendMessage()">Send</button>

  <script>
    function sendMessage() {
      let userInput = document.getElementById("userInput").value;
      if(userInput === '') return;

      let chatbox = document.getElementById("chatbox");
      chatbox.innerHTML += "<div class='user'><strong>You:</strong> " + userInput + "</div>";

      fetch("/get?msg=" + userInput)
        .then(response => response.json())
        .then(data => {
          chatbox.innerHTML += "<div class='bot'><strong>Bot:</strong> " + data.response + "</div>";
          chatbox.scrollTop = chatbox.scrollHeight;
        });

      document.getElementById("userInput").value = "";
    }
  </script>
</body>
</html>
"""

@app.route("/")
def home():
    return render_template_string(html_template)

@app.route("/get")
def get_bot_response():
    user_msg = request.args.get('msg')
    return jsonify(response=chatbot_response(user_msg))

app.run()

Running on http://127.0.0.1:5000/
* Running on http://<random-subdomain>.ngrok.io

